```
curl http://127.0.0.1:8080/predictions/${MODEL_NAME} -T 3dogs.jpg -o 3dogs_mask.png
```

In [74]:
import requests
import json
from PIL import Image
import base64
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [75]:
test_image = "/Users/tseo/Documents/Github/viai-serving/DATA/dent/valid/images/20190505_4365_22799772_41d86711015ea27da507aeb173247702.jpg"
# test_image =  "/Users/tseo/Documents/Github/viai-serving/DATA/dent/test/images/20190227_10535_20402100_8621821f5ed8c37bc0b8e6e0efc9cc0a.jpg"
# test_image = "/Users/tseo/Documents/Github/viai-serving/DATA/dent/train/images/20190421_5831_22190653_95e51a9df25733efb180707d5cfcb9fb.jpg"
# test_image = "/Users/tseo/Documents/Github/viai-serving/DATA/dent/train/images/20190519_7641_23561154_4266de84ff604bd4736d9c536079ba70.jpg"
# test_image = "/Users/tseo/Documents/Github/viai-serving/DATA/dent/train/images/20190505_15096_22780699_4d37042b5290c89c6b08765182d883a1.jpg"
# save_dir = "/Users/tseo/Documents/Github/viai-serving/RESULT/dent"

In [76]:
file_name = test_image.split('/')[-1]
file_name

'20190505_4365_22799772_41d86711015ea27da507aeb173247702.jpg'

In [77]:
import base64
with open(test_image, 'rb') as img:
    base64_string1 = base64.b64encode(img.read())

In [78]:

# raw = cv2.imread(test_image)
# raw = base64.b64encode(test_image)

data = {'name': file_name, 'body' : base64_string1}

In [79]:
# raw

In [80]:
# data = json.dumps(data)
# data

In [81]:
# data = {}
res = requests.post("http://0.0.0.0:9000/predictions/dent", data=data)

In [82]:
res

<Response [200]>

In [83]:
result = res.content
# result
print(type(result))
# img_np = cv2.imdecode(result, cv2.CV_LOAD_)

<class 'bytes'>


In [84]:
import sys
print(sys.getsizeof(result))

174


In [85]:
data_str = result.decode('utf8')
print(data_str)

{
  "class": "dent",
  "mask": "masks-dent/20190505_4365_22799772_41d86711015ea27da507aeb173247702.png",
  "confidence": 0.5924433390893045
}


In [86]:
data_dict = json.loads(data_str)
print(type(data_dict))

<class 'dict'>


In [87]:
data_dict["mask"][0]

'm'

In [88]:
mask_test = np.asarray(data_dict["mask"], dtype="uint8") # list to numpy
mask_test = mask_test * 255 # 1 -> 255
stacked_img = np.stack((mask_test,) * 3, axis=-1) # 1ch -> 3ch
plt.imshow(stacked_img)
plt.show()

ValueError: invalid literal for int() with base 10: 'masks-dent/20190505_4365_22799772_41d86711015ea27da507aeb173247702.png'

In [ ]:
import torch
import torch.nn as nn

m = nn.Softmax(dim=0)
input = torch.rand(2, 3, 4)
print(input.shape)
print(input)

torch.Size([2, 3, 4])
tensor([[[0.0557, 0.9546, 0.3029, 0.9067],
         [0.4793, 0.5003, 0.3799, 0.7595],
         [0.2673, 0.9759, 0.6214, 0.7868]],

        [[0.7370, 0.7230, 0.7914, 0.6269],
         [0.7531, 0.9870, 0.0945, 0.3634],
         [0.5245, 0.3485, 0.8744, 0.6707]]])


In [ ]:
output = m(input)
print(output[0])
print(output[1])
print(output.shape)

tensor([[0.3360, 0.5577, 0.3802, 0.5695],
        [0.4320, 0.3807, 0.5709, 0.5978],
        [0.4361, 0.6519, 0.4371, 0.5290]])
tensor([[0.6640, 0.4423, 0.6198, 0.4305],
        [0.5680, 0.6193, 0.4291, 0.4022],
        [0.5639, 0.3481, 0.5629, 0.4710]])
torch.Size([2, 3, 4])


In [ ]:
mask = torch.Tensor([[0, 1],
                    [1, 1],
                    [0, 0]])
print(mask.shape)

torch.Size([3, 2])


In [ ]:
float(sum(mask[mask > 0])) / np.count_nonzero(np.asarray(mask))

1.0

In [ ]:
s = int(sum(mask[mask == 1]))
s

3

In [ ]:
res = output[1] * mask
print(np.count_nonzero(np.asarray(res != 0)))
res = float(sum(res[res != 0]))
res / s

RuntimeError: The size of tensor a (4) must match the size of tensor b (2) at non-singleton dimension 1

In [ ]:
print(res)

1.544107437133789
